In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
from scipy.io import loadmat

from dash import Dash, html, dcc, Input, Output, callback
import pandas as pd
import plotly.express as px

from turb.lst import lst

In [2]:
mat_f = r'./data/LST_template.mat'
mat_data = loadmat(mat_f)

In [3]:
channel_lst = lst(384, 2)

df = pd.DataFrame(
   dict(
      eigvals = channel_lst.omega,
      eigvecs = [channel_lst.q[:, ind] for ind in range(channel_lst.omega.shape[0])],
      eig_ind = range(channel_lst.omega.shape[0]),
   )
)

Rayleigh number is 0.0


In [4]:
mat_data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'A', 'B', 'omega', 'q'])

In [5]:

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)


app.layout = html.Div([
    html.Div([
        html.Div([
            "Layer Number: ",
            dcc.Input(id='layer_number', value=384, type='number')
        ])]),
    
        html.Div([
            dcc.Graph(
                id='eigenvals_plot',
                clickData={'points': [{'pointNumber': 0}]}
            )
        ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
        
        html.Div([
            dcc.Graph(
                id='eigenvector_plot',
            )
        ], style={'display': 'inline-block', 'width': '49%'})

])

def create_eigvec_plot(y_phys, eigvec, title):
    fig = px.scatter(x=y_phys, y=eigvec[-y_phys.shape[0]:])
    fig.update_traces(mode='lines+markers')

    fig.update_xaxes(showgrid=False)

    fig.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                       xref='paper', yref='paper', showarrow=False, align='left',
                       text=title)

    fig.update_layout(height=225, margin={'l': 20, 'b': 30, 'r': 10, 't': 10})
    
    return fig

@callback(
    Output('eigenvals_plot', 'figure'),
    Input('layer_number', 'value')
)
def update_graph(layer_number, ):
    channel_lst = lst(layer_number, 2)
    fig = px.scatter(x=np.real(channel_lst.omega), y = np.imag(channel_lst.omega), hover_name=df["eig_ind"])
    return fig


@callback(
    Output('eigenvector_plot', 'figure'),
    Input('eigenvals_plot', 'clickData')
)
def update_graph(clickData):
    eig_ind = clickData["points"][0]['pointNumber']
    fig = px.scatter(channel_lst.y_phys[:, 0], channel_lst.q[-channel_lst.y_phys.shape[0]:, eig_ind])
    return fig


In [6]:
app.run(debug=True)

Rayleigh number is 0.0
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File /opt/anaconda3/envs/turb/lib/python3.10/site-packages/dash/_callback.py:491, in register_callback.<locals>.wrap_func.<locals>.add_context(
    *args=({'points': [{'pointNumber': 0}]},),
    **kwargs={}
)
    490 try:
--> 491     jsonResponse = to_json(response)
        response = {'multi': True, 'response': defaultdict(<class 'dict'>, {'eigenvector_plot': {'figure': Figure({
    'data': [{'hovertemplate': 'x=%{x}<br>index=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'h',
              'showlegend': False,
              'type': 'scatter',
              'x': array([0j, 0j, 0j, ..., 0j, 0j, 0j], dtype=object),
              'xaxis': 'x',
             